In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def filter_location(location):
    result = location.split(",")
    if len(result) > 1:
        return result[1][1:] #lấy phần tử có index là 1, bỏ ký tự trắng đầu tiên (Houston, TX => lấy đc TX)
    else:
        return location

data = pd.read_excel("week8/job_dataset.ods", engine="odf", dtype="str")
data = data.dropna(axis=0)
data["location"] = data["location"].apply(filter_location)

target = "career_level"

x = data.drop(target, axis=1)
y = data[target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2024)



In [5]:
preprocessor =  ColumnTransformer(transformers=[
    ("title", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "title"),
    ("location", OneHotEncoder(handle_unknown='ignore'), ["location"]),
    ("description", TfidfVectorizer(stop_words="english", ngram_range=(1, 2)), "description"),
    ("function", OneHotEncoder(handle_unknown='ignore'), ["function"]),
    ("industry", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "industry"),
])

model = Pipeline(steps=[
    ("pre_processor", preprocessor),
    ("regressor", RandomForestClassifier(random_state=100))
])

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

print(classification_report(y_test, y_predict))

                                      precision    recall  f1-score   support

                      bereichsleiter       0.75      0.03      0.06       188
       director_business_unit_leader       1.00      0.19      0.32        16
                 manager_team_leader       0.64      0.51      0.57       546
senior_specialist_or_project_manager       0.71      0.96      0.82       863
                          specialist       0.00      0.00      0.00         2

                            accuracy                           0.69      1615
                           macro avg       0.62      0.34      0.35      1615
                        weighted avg       0.69      0.69      0.64      1615



/Users/tuantai229/Projects/VietAI-x-CoderSchool/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuantai229/Projects/VietAI-x-CoderSchool/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuantai229/Projects/VietAI-x-CoderSchool/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

In [ ]:
# Bài tập trên lớp, xử lý lỗi `ValueError: Found unknown categories ['South Carolina', 'New Hampshire'] in column 0 during transform`

# Phân tích lỗi:
# - Lỗi này xảy ra với OneHotEncoder khi nó gặp các giá trị trong dữ liệu test mà chưa từng xuất hiện trong dữ liệu training
# - Trong trường hợp này, các địa điểm 'South Carolina' và 'New Hampshire' xuất hiện trong tập test nhưng không có trong tập training
# - Mặc định, OneHotEncoder sẽ báo lỗi khi gặp các categories mới trong quá trình transform

# Cách khắc phục: Thêm tham số handle_unknown='ignore' cho OneHotEncoder:

# preprocessor =  ColumnTransformer(transformers=[
#     ("title", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "title"),
#     ("location", OneHotEncoder(), ["location"]),
#     ("description", TfidfVectorizer(stop_words="english", ngram_range=(1, 2)), "description"),
#     ("function", OneHotEncoder(), ["function"]),
#     ("industry", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "industry"),
# ])
# ===>>
# preprocessor = ColumnTransformer(transformers=[
#     ("title", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "title"),
#     ("location", OneHotEncoder(handle_unknown='ignore'), ["location"]), # Thêm handle_unknown='ignore'
#     ("description", TfidfVectorizer(stop_words="english", ngram_range=(1, 2)), "description"),
#     ("function", OneHotEncoder(handle_unknown='ignore'), ["function"]), # Thêm cho cả function
#     ("industry", TfidfVectorizer(stop_words="english", ngram_range=(1, 1)), "industry"),
# ])

In [8]:
# Cài tiến 1: Xử lý mất cân bằng dữ liệu bằng tham số class_weight='balanced' của RandomForestClassifier

model1 = Pipeline(steps=[
    ("pre_processor", preprocessor),
    ("regressor", RandomForestClassifier(random_state=100, class_weight='balanced'))
])

model1.fit(x_train, y_train)
y_predict1 = model.predict(x_test)

print(classification_report(y_test, y_predict1, zero_division=0))

                                      precision    recall  f1-score   support

                      bereichsleiter       0.64      0.04      0.07       188
       director_business_unit_leader       1.00      0.19      0.32        16
                 manager_team_leader       0.63      0.46      0.53       546
senior_specialist_or_project_manager       0.69      0.96      0.80       863
                          specialist       0.00      0.00      0.00         2

                            accuracy                           0.67      1615
                           macro avg       0.59      0.33      0.34      1615
                        weighted avg       0.66      0.67      0.62      1615



In [12]:
# Cài tiến 2: Xử lý mất cân bằng dữ liệu bằng cách sử dụng over_sampling bằng SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImblearnPipeline

model2 = ImblearnPipeline(steps=[
    ("pre_processor", preprocessor),
    ("smote", SMOTE(random_state=100, k_neighbors=2)), # giảm k_neighbors xuống 2 để tránh lỗi `ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 4, n_samples = 4`. Do SMOTE không thể tạo các mẫu tổng hợp mới vì số lượng neighbors yêu cầu (mặc định là 5) lớn hơn số lượng mẫu có sẵn trong một số classes thiểu số hiện tại là 4
    ("regressor", RandomForestClassifier(random_state=100))
])

model2.fit(x_train, y_train)
y_predict2 = model2.predict(x_test)

print(classification_report(y_test, y_predict2, zero_division=0))

                                      precision    recall  f1-score   support

                      bereichsleiter       0.67      0.09      0.15       188
       director_business_unit_leader       1.00      0.19      0.32        16
                 manager_team_leader       0.62      0.54      0.58       546
senior_specialist_or_project_manager       0.72      0.93      0.82       863
                          specialist       0.00      0.00      0.00         2

                            accuracy                           0.69      1615
                           macro avg       0.60      0.35      0.37      1615
                        weighted avg       0.69      0.69      0.65      1615



In [14]:
# Dùng GridSearchCV tìm params
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [10, 20, None],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['sqrt', 'log2'],
    'regressor__class_weight': ['balanced', 'balanced_subsample']
}

grid_search = GridSearchCV(
    model1,
    param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(x_train, y_train)
print("\nBest parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

y_pred_best = grid_search.predict(x_test)
print("\nKết quả với best parameters:")
print(classification_report(y_test, y_pred_best, zero_division=0))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/Users/tuantai229/Projects/VietAI-x-CoderSchool/venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/tuantai229/Projects/VietAI-x-CoderSchool/venv/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Best parameters: {'regressor__class_weight': 'balanced_subsample', 'regressor__max_depth': 20, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 200}
Best cross-validation score: 0.43067163175896256

Kết quả với best parameters:
                                      precision    recall  f1-score   support

                      bereichsleiter       0.63      0.23      0.34       188
       director_business_unit_leader       1.00      0.19      0.32        16
                 manager_team_leader       0.61      0.58      0.60       546
senior_specialist_or_project_manager       0.76      0.90      0.82       863
                          specialist       0.00      0.00      0.00         2

                            accuracy                           0.71      1615
                           macro avg       0.60      0.38      0.41      1615
                        weighted avg       0.70      0.71      